In [ ]:
# Install the library if you are running this in Colab
!pip install py-adtools


# adtools.py_code 教程：Python 代码解析与操作

`adtools.py_code` 提供了一个强大的 Python 代码解析器，可以将 Python 源代码解析为结构化的组件（如函数、类、代码块）。这使得我们可以轻松地分析、修改和重构代码，特别适合用于处理大语言模型（LLM）生成的代码。

## 核心组件

解析器将代码分解为以下四个主要数据结构：

| 组件 | 描述 | 关键属性 |
|---|---|---|
| **PyProgram** | 代表整个文件。它维护脚本、函数和类的确切顺序。 | `functions`, `classes`, `scripts`, `elements` |
| **PyFunction** | 代表顶级函数或类方法。你可以动态修改其签名、装饰器、文档字符串或函数体。 | `name`, `args`, `body`, `docstring`, `decorator`, `return_type` |
| **PyClass** | 代表类定义。它是方法和类级语句的容器。 | `name`, `bases`, `functions` (methods), `body` |
| **PyCodeBlock** | 代表原始代码段，如导入、全局变量或类内部的特定逻辑块。 | `code` |


## 基础用法

首先，我们将一段包含导入、函数和类的 Python 代码解析为 `PyProgram` 对象。


In [ ]:
from adtools import PyProgram

code = r"""
import ast, numba                 # 这部分将被解析为 PyCodeBlock
import numpy as np

@numba.jit()                      # 这部分将被解析为 PyFunction
def function(arg1, arg2=True):     
    '''Docstring.
    This is a function.
    '''
    if arg2:
        return arg1 * 2
    else:
        return arg1 * 4

@some.decorators()                # 这部分将被解析为 PyClass
class PythonClass(BaseClass):
    '''Docstring.'''
    # Comments
    class_var1 = 1                # 这部分将被解析为 PyCodeBlock
    class_var2 = 2                # 并放置在 PyClass.body 中

    def __init__(self, x):        # 这部分将被解析为 PyFunction
        self.x = x                # 并放置在 PyClass.functions 中

    def method1(self):
        '''Docstring.
        This is a class method.
        '''
        return self.x * 10

    @some.decorators()
    def method2(self, x, y):
        return x + y + self.method1(x)
    
    @some.decorators(100)  
    class InnerClass:             # 这部分将被解析为 PyCodeBlock
        '''Docstring.'''
        def __init__(self):
            ... # 并放置在 PyClass.body 中

if __name__ == '__main__':        # 这部分将被解析为 PyCodeBlock
    res = function(1)
    print(res)
    res = PythonClass().method2(1, 2)
"""

# 解析代码
p = PyProgram.from_text(code)
print("代码解析成功！")


## 访问和检查组件

解析后，我们可以访问程序中的各个组件。例如，查看所有的类、函数，或者查看特定类的方法。


In [ ]:

# 打印整个程序结构（重构后的代码）
print("--- 重构后的完整代码 ---")
print(p)
print("-------------------------------------")

# 访问第一个类中的第二个方法 (method1)
print(f"类名: {p.classes[0].name}")
print(f"方法名: {p.classes[0].functions[1].name}")
print(f"方法文档: {p.classes[0].functions[1].docstring}")
print("-------------------------------------")

# 访问第一个类中的第三个方法 (method2) 的装饰器
print(f"方法 {p.classes[0].functions[2].name} 的装饰器: {p.classes[0].functions[2].decorator}")
print("-------------------------------------")

# 访问全局函数
print(f"全局函数名: {p.functions[0].name}")


## 修改代码

`adtools` 的强大之处在于能够以编程方式修改代码。你可以更改函数名、移除装饰器、修改文档字符串，甚至替换函数体。


In [ ]:

# 获取全局函数
func = p.functions[0]

# 修改函数名
func.name = "renamed_function"

# 移除装饰器
func.decorator = None

# 修改文档字符串
func.docstring = "这是一个被修改过的函数。"

# 打印修改后的函数
print(func)


## LLM 提示词构建应用

在基于 LLM 的算法设计中，我们经常需要处理生成的代码。例如，从 LLM 输出中提取函数，重命名它们，或者标准化文档字符串，然后再将其作为提示词的一部分反馈给 LLM。


In [ ]:
from adtools import PyFunction

# 假设 LLM 生成了两个版本的交叉变异算法
llm_output_1 = '''
def crossover(p1, p2):
    """Single point crossover."""
    point = len(p1) // 2
    return p1[:point] + p2[point:], p2[:point] + p1[point:]
'''

llm_output_2 = """
def crossover_op(parent_a, parent_b):
    # This is a uniform crossover
    mask = [True, False] * (len(parent_a) // 2)
    return [a if m else b for a, b, m in zip(parent_a, parent_b, mask)]
"""

# 从文本中提取函数
func_v1 = PyFunction.extract_first_function_from_text(llm_output_1)
func_v2 = PyFunction.extract_first_function_from_text(llm_output_2)

# --- 修改逻辑 ---

# 1. 标准化命名：重命名为 v1 和 v2
func_v1.name = "crossover_v1"
func_v2.name = "crossover_v2"

# 2. 文档字符串管理
# 为 v1 设置特定的文档字符串
func_v1.docstring = "Variant 1: Implementation of Single Point Crossover."

# 为 v2 移除文档字符串（例如为了节省 Token）
func_v2.docstring = None

# --- 构建提示词 ---
prompt = "Here are the two crossover algorithms currently in the population:"
prompt += str(func_v1) + "\n\n"
prompt += str(func_v2) + "\n\n"
prompt += "Please generate a v3 that combines the best features of both."

print(prompt)
